In [11]:
!pip install langchain-community
!pip install langchain

In [12]:
from langchain.document_loaders.csv_loader import CSVLoader

In [13]:
!pip install llama-index

In [14]:
%pip install llama-index-llms-mistralai

In [15]:
%pip install llama-index-embeddings-mistralai

In [16]:
!pip install --upgrade llama-index

In [17]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,ServiceContext
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.core.query_engine import RetrieverQueryEngine

In [25]:
loader = CSVLoader(file_path='/content/property_details_10.csv', encoding='utf-8', csv_args={'delimiter': ','})

# Load data
reader = SimpleDirectoryReader(input_files=['/content/property_details_10.csv'])
documents = reader.load_data()
documents

[Document(id_='0d8ba714-5db6-4746-8fe0-c49b7df73011', embedding=None, metadata={'file_path': '/content/property_details_10.csv', 'file_name': 'property_details_10.csv', 'file_type': 'text/csv', 'file_size': 25768, 'creation_date': '2024-06-22', 'last_modified_date': '2024-06-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='0, 6596316, Welcome to this beautiful sanctuary located in the highly coveted Ashford Forest Lake Section in the Energy Corridor. Step into a private retreat just steps away from Ashford Lake. Natural lights floods into the first floor living, complete with updated gourmet kitchen, high end appliances, wine bar, study, formal living and dining room. Upstairs you will find the spacious Primary bedroom and en-suite bathroom with dou

In [19]:
data = loader.load()

In [20]:
data

[Document(page_content=": 0\nzpid: 6596316\ndescription: Welcome to this beautiful sanctuary located in the highly coveted Ashford Forest Lake Section in the Energy Corridor. Step into a private retreat just steps away from Ashford Lake. Natural lights floods into the first floor living, complete with updated gourmet kitchen, high end appliances, wine bar, study, formal living and dining room. Upstairs you will find the spacious Primary bedroom and en-suite bathroom with double vanities, walk-in closet, soaking tub and walk-in shower. The exterior offers an exceptionally large backyard with private pool, ravine and lake access and side yard perfect for relaxing and entertaining! Additional Amenities include access to two neighborhood pools, tennis courts, and the scenic Terry Hershey Park Hike and Bike Trail. Nestled in the heart of the City, this home offers a serene oasis, surrounded by natural beauty and lush privacy, providing an idyllic retreat amidst urban convenience.\nhomeType:

In [21]:
from llama_index.core import PromptTemplate
prompt_template = """\ Your a helpful assisstant for creating an marketing pitch for real estate property\
For the given property details and target audience generate a tailored and compelling pitch.\

property details: {property_details}
target audience: {target_audience}

The response strictly should be in JSON format with 3 to 4 sentences.\
For example:
{{
  "Reponse: {{"pitch":"Experience the pinnacle of luxury at Ocean Breeze Residences in Newport Beach, CA. Designed for dynamic professionals, our stunning beachfront homes offer private beach access, infinity pools, and a state-of-the-art spa. Choose from exquisite 2, 3, or 4-bedroom residences, priced from $1.8M to $4M, each featuring gourmet kitchens and high-end finishes. Elevate your lifestyle in a community where sophistication meets coastal serenity"}}
"""

template_var_mapping = {"property_str":"property_details", "target_str":"target_audience"}

prompt_tmpl = PromptTemplate(
    prompt_template, template_var_mappings=template_var_mapping
)

property_details = {
  "Name": "Mountain Horizon Cabins",
  "Location": "Aspen, CO",
  "Property Type": "Luxury Cabins",
  "Availability": "3 Bed, 2 Bath | 4 Bed, 3 Bath | 5 Bed, 4 Bath",
  "Price Range": "$2M - $5M",
  "Amenities": ["Ski-In/Ski-Out Access", "Private Hot Tubs", "Stone Fireplaces", "Gourmet Kitchens with Chef-Grade Appliances", "Dedicated Ski Storage Rooms", "Home Automation Systems"],
  "Description": "Experience the ultimate mountain retreat with our luxurious cabins in Aspen. Perfect for winter sports enthusiasts and nature lovers, Mountain Horizon Cabins combine rustic charm with modern convenience, offering stunning views of the Rockies and direct access to trails and ski slopes."
}
targeted_group = "age between 25 and 30"

fmt_prompt = prompt_tmpl.format(property_details=property_details, target_audience=targeted_group)
print(fmt_prompt)


\ Your a helpful assisstant for creating an marketing pitch for real estate propertyFor the given property details and target audience generate a tailored and compelling pitch.
property details: {'Name': 'Mountain Horizon Cabins', 'Location': 'Aspen, CO', 'Property Type': 'Luxury Cabins', 'Availability': '3 Bed, 2 Bath | 4 Bed, 3 Bath | 5 Bed, 4 Bath', 'Price Range': '$2M - $5M', 'Amenities': ['Ski-In/Ski-Out Access', 'Private Hot Tubs', 'Stone Fireplaces', 'Gourmet Kitchens with Chef-Grade Appliances', 'Dedicated Ski Storage Rooms', 'Home Automation Systems'], 'Description': 'Experience the ultimate mountain retreat with our luxurious cabins in Aspen. Perfect for winter sports enthusiasts and nature lovers, Mountain Horizon Cabins combine rustic charm with modern convenience, offering stunning views of the Rockies and direct access to trails and ski slopes.'}
target audience: age between 25 and 30

The response strictly should be in JSON format with 3 to 4 sentences.For example:
{
  "

In [23]:
!pip install --upgrade llama-index

In [26]:
llm = MistralAI(api_key= "api_key", model="mistral-small")
embed_model = MistralAIEmbedding(model_name="mistral-embed", api_key="api_key")
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

# Create vector store index
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

# Create query engine
query_engine = index.as_query_engine(similarity_top_k=2)

formatted_query = prompt_template.format(property_details=property_details, target_audience=targeted_group)
response = query_engine.query(
   formatted_query
)
print(str(response))

<ipython-input-26-d84844140104>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)


{
  "Response": {
    "pitch": "Discover a new level of luxury in the heart of Aspen with Mountain Horizon Cabins. Ideal for adventurous spirits and young professionals, these 3, 4, and 5-bedroom cabins offer ski-in/ski-out access, private hot tubs, and gourmet kitchens with chef-grade appliances. Priced between $2M - $5M, indulge in the perfect blend of rustic charm and modern convenience, all while enjoying stunning views of the Rockies and direct access to trails and ski slopes."
  }
}


In [28]:
import json
json_reponse = json.loads(str(response))
json_reponse['Response']

{'Response': {'pitch': 'Discover a new level of luxury in the heart of Aspen with Mountain Horizon Cabins. Ideal for adventurous spirits and young professionals, these 3, 4, and 5-bedroom cabins offer ski-in/ski-out access, private hot tubs, and gourmet kitchens with chef-grade appliances. Priced between $2M - $5M, indulge in the perfect blend of rustic charm and modern convenience, all while enjoying stunning views of the Rockies and direct access to trails and ski slopes.'}}


In [10]:
from llama_index.core import PromptTemplate
prompt_template = """\ Your a helpful assisstant for creating an marketing pitch for real estate property\
For the given property details and target audience generate a tailored and compelling pitch.\

property details: {property_details}
target audience: {target_audience}

The response strictly should be in JSON format with 3 to 4 sentences.\
For example:
{{
  "Reponse: {{"pitch":"Experience the pinnacle of luxury at Ocean Breeze Residences in Newport Beach, CA. Designed for dynamic professionals, our stunning beachfront homes offer private beach access, infinity pools, and a state-of-the-art spa. Choose from exquisite 2, 3, or 4-bedroom residences, priced from $1.8M to $4M, each featuring gourmet kitchens and high-end finishes. Elevate your lifestyle in a community where sophistication meets coastal serenity"}}
"""

template_var_mapping = {"property_str":"property_details", "target_str":"target_audience"}

prompt_tmpl = PromptTemplate(
    prompt_template, template_var_mappings=template_var_mapping
)

property_details = {
  "Name": "Mountain Horizon Cabins",
  "Location": "Aspen, CO",
  "Property Type": "Luxury Cabins",
  "Availability": "3 Bed, 2 Bath | 4 Bed, 3 Bath | 5 Bed, 4 Bath",
  "Price Range": "$2M - $5M",
  "Amenities": ["Ski-In/Ski-Out Access", "Private Hot Tubs", "Stone Fireplaces", "Gourmet Kitchens with Chef-Grade Appliances", "Dedicated Ski Storage Rooms", "Home Automation Systems"],
  "Description": "Experience the ultimate mountain retreat with our luxurious cabins in Aspen. Perfect for winter sports enthusiasts and nature lovers, Mountain Horizon Cabins combine rustic charm with modern convenience, offering stunning views of the Rockies and direct access to trails and ski slopes."
}
targeted_group = "age between 25 and 30"

fmt_prompt = prompt_tmpl.format(property_details=property_details, target_audience=targeted_group)
print(fmt_prompt)


\ Your a helpful assisstant for creating an marketing pitch for real estate propertyFor the given property details and target audience generate a tailored and compelling pitch.
property details: {'Name': 'Mountain Horizon Cabins', 'Location': 'Aspen, CO', 'Property Type': 'Luxury Cabins', 'Availability': '3 Bed, 2 Bath | 4 Bed, 3 Bath | 5 Bed, 4 Bath', 'Price Range': '$2M - $5M', 'Amenities': ['Ski-In/Ski-Out Access', 'Private Hot Tubs', 'Stone Fireplaces', 'Gourmet Kitchens with Chef-Grade Appliances', 'Dedicated Ski Storage Rooms', 'Home Automation Systems'], 'Description': 'Experience the ultimate mountain retreat with our luxurious cabins in Aspen. Perfect for winter sports enthusiasts and nature lovers, Mountain Horizon Cabins combine rustic charm with modern convenience, offering stunning views of the Rockies and direct access to trails and ski slopes.'}
target audience: age between 25 and 30

The response strictly should be in JSON format with 3 to 4 sentences.For example:
{
  "